In [28]:
import torch
import torch.nn as nn
import numpy as np
from scipy.io import loadmat
from tqdm import tqdm
from pyDOE import lhs
import torch.optim as optim

In [29]:
# class PINN(nn.Module):
#     def __init__(self, input_size, hidden_size, output_size):
#         super(PINN, self).__init__()
#         self.layers = nn.ModuleList(
#             [
#                 nn.Linear(input_size if i == 0 else hidden_size, hidden_size)
#                 if i % 2 == 0
#                 else nn.Tanh()
#                 for i in range(10)
#             ]
#         )
#         self.layers.append(nn.Linear(hidden_size, output_size))
        
#         # Trainable parameter for the wave number squared (k^2)
#         self.k2 = nn.Parameter(torch.tensor([30.0], dtype=torch.float32, device="cuda"))

#         # Optimizer
#         self.optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
#         self.optimizer.param_groups[0]["params"].append(self.k2)
        
#         self.loss = nn.MSELoss()

#     def forward(self, x):
#         for layer in self.layers:
#             x = layer(x)
#         return x

#     def loss_fn(self, x, u):
#         u_pred = self.forward(x)
#         return self.loss(u_pred, u)

#     # def residual_loss(self, xtrain):
#     #     g = xtrain.clone()
#     #     g.requires_grad = True
#     #     u_pred = self.forward(g)
        
#     #     # Compute gradients
#     #     u_grad = torch.autograd.grad(
#     #         u_pred, g, torch.ones_like(u_pred), retain_graph=True, create_graph=True
#     #     )[0]
#     #     u_lap = torch.autograd.grad(
#     #         u_grad, g, torch.ones_like(u_grad), create_graph=True
#     #     )[0].sum(dim=1, keepdim=True)
        
#     #     # Residual form of the Helmholtz equation
#     #     residual = u_lap + self.k2 * u_pred
#     #     return self.loss(residual, torch.zeros_like(residual))
#     #     THIS IS VERY STUPID OMFG LET'S TRY SOMETHING ELSE CUZ THE GODS HATE ME AND LIFE IS AN ENDLESS PIT OF DESPAIR THAT IS VOID OF ALL HAPPINESS
    
#     def residual_loss(self, xtrain, fhat):
#         g = xtrain.clone()
#         g.requires_grad = True
#         u_pred = self.forward(g)
        
#         # Compute gradients
#         u_grad = torch.autograd.grad(
#             u_pred, g, torch.ones_like(u_pred), create_graph=True, retain_graph=True
#         )[0]
#         u_xx = torch.autograd.grad(
#             u_grad[:, [0]], g, torch.ones_like(u_grad[:, [0]]), create_graph=True
#         )[0][:, [0]]
#         u_tt = torch.autograd.grad(
#             u_grad[:, [1]], g, torch.ones_like(u_grad[:, [1]]), create_graph=True
#         )[0][:, [1]]
        
#         # Residual calculation
#         residual = u_xx + u_tt + self.k2 * u_pred - fhat
#         return torch.mean(residual**2)  # Mean squared error for the residual


#     def total_loss(self, xtrain, utrain):
#         alpha_female = 10.0
#         data_loss = self.loss_fn(xtrain, utrain)  # Match observed data
#         physics_loss = self.residual_loss(xtrain)  # Enforce governing equations
#         return data_loss + alpha_female* physics_loss

#     def train_model(self, xtrain, utrain, epochs=10000):
#         for epoch in tqdm(range(epochs)):
#             self.optimizer.zero_grad()
#             loss = self.total_loss(xtrain, utrain)
#             loss.backward()
#             self.optimizer.step()
            
#             # Logging
#             if epoch % 1000 == 0:
#                 print(
#                     f"Epoch {epoch}, Loss {loss.item()}, "
#                     f"k^2 (Wave Number Squared) {self.k2.item()}"
#                 )

In [48]:
# again.

class PINN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(PINN, self).__init__()
        self.layers = nn.ModuleList(
            [
                nn.Linear(input_size if i == 0 else hidden_size, hidden_size)
                if i % 2 == 0
                else nn.Tanh()
                for i in range(10)
            ]
        )
        self.layers.append(nn.Linear(hidden_size, output_size))
        
        # Trainable parameter for the wave number squared (k^2)
        self.k2 = nn.Parameter(torch.tensor([1.0], dtype=torch.float32, device="cuda")) 

        

        # Optimizer
        self.optimizer = torch.optim.Adam(self.parameters(), lr=1e-2)  # Larger learning rate
        self.scheduler = torch.optim.lr_scheduler.StepLR(self.optimizer, step_size=1000, gamma=0.9)  # Scheduler to reduce LR

        self.loss = nn.MSELoss()

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

    def loss_fn(self, x, u):
        u_pred = self.forward(x)
        return self.loss(u_pred, u)

    def residual_loss(self, xtrain, fhat):
        # Physics-informed loss based on the Helmholtz equation
        g = xtrain.clone()
        g.requires_grad = True
        u_pred = self.forward(g)
        
        # Compute the gradients for second derivatives (u_xx and u_tt)
        u_grad = torch.autograd.grad(
            u_pred, g, torch.ones_like(u_pred), create_graph=True, retain_graph=True
        )[0]
        u_xx = torch.autograd.grad(
            u_grad[:, [0]], g, torch.ones_like(u_grad[:, [0]]), create_graph=True
        )[0][:, [0]]
        u_tt = torch.autograd.grad(
            u_grad[:, [1]], g, torch.ones_like(u_grad[:, [1]]), create_graph=True
        )[0][:, [1]]
        
        # Residual calculation for Helmholtz equation: u_xx + u_tt + k^2 * u = fhat
        residual = u_xx + u_tt + self.k2 * u_pred - fhat
        return torch.mean(residual**2)  # Mean squared error for the residual

    def total_loss(self, xtrain, utrain):
        alpha_female = 10.0
        data_loss = self.loss_fn(xtrain, utrain)  # Match observed data
        physics_loss = self.residual_loss(xtrain, utrain)  # Enforce governing equations
        return data_loss + alpha_female * physics_loss

    def train_model(self, xtrain, utrain, epochs=10000):
        for epoch in tqdm(range(epochs)):
            self.optimizer.zero_grad()
            loss = self.total_loss(xtrain, utrain)
            loss.backward()
            self.optimizer.step()
            self.scheduler.step()
            
            # Logging
            if epoch % 1000 == 0:
                print(
                    f"Epoch {epoch}, Loss {loss.item()}, "
                    f"k^2 (Wave Number Squared) {self.k2.item()}"
                )

In [31]:
u = np.load("/home/pes1ug22am100/Documents/Research and Experimentation/NoisyICML/pinns-inverse/Helmholtz/helmholtz_solution.npy")  # BOB THE BUILDER
x = np.load("/home/pes1ug22am100/Documents/Research and Experimentation/NoisyICML/pinns-inverse/Helmholtz/x_coordinate.npy")  # BOB THE BUILDER
t = np.load("/home/pes1ug22am100/Documents/Research and Experimentation/NoisyICML/pinns-inverse/Helmholtz/t_coordinate.npy")[:-1]

In [32]:
x = torch.tensor(x, dtype=torch.float32)
t = torch.tensor(t, dtype=torch.float32)
u = torch.tensor(u, dtype=torch.float32)

In [33]:
X, T = np.meshgrid(x, t)
xtrue = np.hstack((X.flatten()[:, None], T.flatten()[:, None]))
utrue = u.flatten()[:, None]

In [49]:
idx = np.random.choice(xtrue.shape[0], 10000, replace=False)
xtrain = xtrue[idx, :]
utrain = utrue[idx, :]

device = torch.device("cuda")
Xtrain = torch.tensor(xtrain, dtype=torch.float32, device=device)
Utrain = torch.tensor(utrain, dtype=torch.float32, device=device)

model = PINN(input_size=2, hidden_size=20, output_size=1).to(device)
model.train_model(Xtrain, Utrain, epochs=10000)


/tmp/ipykernel_8086/181255850.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Utrain = torch.tensor(utrain, dtype=torch.float32, device=device)
  0%|          | 28/10000 [00:00<01:12, 138.04it/s]

Epoch 0, Loss 3.0966885089874268, k^2 (Wave Number Squared) 0.9900000095367432


 10%|█         | 1033/10000 [00:06<00:53, 166.85it/s]

Epoch 1000, Loss 2.555675983428955, k^2 (Wave Number Squared) 0.9473637342453003


 20%|██        | 2032/10000 [00:12<00:51, 154.76it/s]

...

 80%|████████  | 8021/10000 [00:50<00:12, 161.03it/s]

Epoch 8000, Loss 2.123868227005005, k^2 (Wave Number Squared) 0.7574743032455444


 90%|█████████ | 9022/10000 [00:56<00:06, 160.65it/s]

Epoch 9000, Loss 2.097508430480957, k^2 (Wave Number Squared) 1.1546372175216675


100%|██████████| 10000/10000 [01:02<00:00, 159.62it/s]
